In [6]:
import pandas as pd
path = '../dataset/'
prefix = "Fecom Inc"

customers_df = pd.read_csv(f'{path}/{prefix} Customer List.csv', sep=';')
geolocations_df = pd.read_csv(f'{path}/{prefix} Geolocations.csv', sep=';')
order_items_df = pd.read_csv(f'{path}/{prefix} Order Items.csv', sep=';')
order_payments_df = pd.read_csv(f'{path}/{prefix} Order Payments.csv', sep=';')
orders_df = pd.read_csv(f'{path}/{prefix} Orders.csv', sep=';')
products_df = pd.read_csv(f'{path}/{prefix} Products.csv', sep=';')
sellers_df = pd.read_csv(f'{path}/{prefix} Sellers List.csv', sep=';')
reviews_df = pd.read_csv(f'{path}/Fecom_Inc_Order_Reviews_No_Emojis.csv', sep=';')

customers_df

C:\Users\Familia\AppData\Local\Temp\ipykernel_15944\1127542560.py:6: DtypeWarning: Columns (0,1,2,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  geolocations_df = pd.read_csv(f'{path}/{prefix} Geolocations.csv', sep=';')


,Customer_Trx_ID,Subscriber_ID,Subscribe_Date,First_Order_Date,Customer_Postal_Code,Customer_City,Customer_Country,Customer_Country_Code,Age,Gender
0,1e959e1f5920cba43823fa9f95673b83,9765e039028279fd2e60bb620a451526,2023-07-08,2023-07-09,FR-75005,Paris,France,FR,29,Male
1,9877437582f263da7d7e30a90c57b8bb,a75e134e7eb6f96e2b0c716ac2a82efb,2024-03-23,2024-04-11,PL-00-001,Warsaw,Poland,PL,38,Male
2,fa6fbbb2080646acae977bf2e44af98b,2fdac27295500e820e43910e9a0aa8d8,2023-05-12,2023-06-01,NL-1012,Amsterdam,Netherlands,NL,35,Female
3,a4c9ff14ae7620126461ca55f36a76ea,e9ab8fd8ea96c85be2714c7f573fb7cd,2023-04-16,2023-04-26,IT-00144,Rome,Italy,IT,62,Male
4,93d5e378ae2f72a07db704f1f6716a7e,6384e6a7b021717616f25625b4bb0bf9,2023-05-26,2023-06-27,NL-1011,Amsterdam,Netherlands,NL,19,Male
...,...,...,...,...,...,...,...,...,...,...
102722,NaN,fb53848e076b4863a0e3ce0750bde66e,2024-09-14,NaN,GB-W1,London,United Kingdom,GB,34,Male
102723,NaN,40ed38022bce40ad9f79dd1b53a2499b,2023-03-27,NaN,DE-10115,Berlin,Germany,DE,34,Female
102724,NaN,ef33bf0a9b1742a3a0944fb603e81c06,2024-09-17,NaN,IT-20125,Milan,Italy,IT,42,Female
102725,NaN,abb296dcaa8f4961a6ea6681eff35e2c,2024-09-02,NaN,GB-S2,Sheffield,United Kingdom,GB,27,Female


In [7]:
items_reviews = pd.merge(reviews_df, order_items_df, on='Order_ID')

# Unir con productos para obtener características
df = pd.merge(items_reviews, products_df, on='Product_ID')

# Mostrar preview
df.head()

,Review_ID,Order_ID,Review_Score,Review_Comment_Title_En,Review_Comment_Message_En,Review_Creation_Date,Review_Answer_Timestamp,Order_Item_ID,Product_ID,Seller_ID,Shipping_Limit_Date,Price,Freight_Value,Product_Category_Name,Product_Weight_Gr,Product_Length_Cm,Product_Height_Cm,Product_Width_Cm
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2024-01-18 00:00,2024-01-18 21:46,1,fd25ab760bfbba13c198fa3b4f1a0cd3,6d803cb79cc31c41c4c789a75933b3c7,2024-01-18 15:47,185.00,13.63,Sports_Leisure,1300.0,30.0,30.0,35.0
1,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2024-01-18 00:00,2024-01-18 21:46,2,fd25ab760bfbba13c198fa3b4f1a0cd3,6d803cb79cc31c41c4c789a75933b3c7,2024-01-18 15:47,185.00,13.63,Sports_Leisure,1300.0,30.0,30.0,35.0
2,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2024-03-10 00:00,2024-03-11 03:05,1,be0dbdc3d67d55727a65d4cd696ca73c,8e6d7754bc7e0f22c96d255ebda59eba,2024-03-06 12:48,79.79,8.30,Computers_Accessories,245.0,19.0,14.0,14.0
3,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2024-02-17 00:00,2024-02-18 14:36,1,d1c427060a0f73f6b889a5c7c61f2ac4,a1043bafd471dff536d0c462352beb48,2024-02-08 10:31,149.00,45.12,Computers_Accessories,6550.0,20.0,20.0,20.0
4,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,I received it well before the stipulated deadl...,2023-04-21 00:00,2023-04-21 22:02,1,52c80cedd4e90108bf4fa6a206ef6b03,a1043bafd471dff536d0c462352beb48,2023-04-13 17:55,179.99,42.85,Garden_Tools,7650.0,20.0,20.0,20.0


In [8]:
df['calificacion_buena'] = df['Review_Score'].apply(lambda x: 1 if x >= 4 else 0)



# Seleccionar características del producto
features = [
    'Product_Weight_Gr', 'Product_Length_Cm', 'Product_Height_Cm',
    'Product_Width_Cm', 'Product_Category_Name'
]

# Eliminar nulos
df = df[features + ['calificacion_buena']].dropna()



df.head()

,Product_Weight_Gr,Product_Length_Cm,Product_Height_Cm,Product_Width_Cm,Product_Category_Name,calificacion_buena
0,1300.0,30.0,30.0,35.0,Sports_Leisure,1
1,1300.0,30.0,30.0,35.0,Sports_Leisure,1
2,245.0,19.0,14.0,14.0,Computers_Accessories,1
3,6550.0,20.0,20.0,20.0,Computers_Accessories,1
4,7650.0,20.0,20.0,20.0,Garden_Tools,1


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Separar features y variable objetivo
X = df[[
    'Product_Weight_Gr', 'Product_Length_Cm', 'Product_Height_Cm',
    'Product_Width_Cm'
]]
y = df['calificacion_buena']

# Entrenar/test split
# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar modelo
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Predicciones y evaluación
y_pred = model.predict(X_test)


              precision    recall  f1-score   support

           0       0.45      0.16      0.23      5504
           1       0.77      0.94      0.85     16646

    accuracy                           0.74     22150
   macro avg       0.61      0.55      0.54     22150
weighted avg       0.69      0.74      0.69     22150

